In [1]:
# !pip install datasets
# !pip install transformers
# !pip install evaluate
# !pip install transformers[torch]

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import create_optimizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers.keras_callbacks import KerasMetricCallback
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

In [3]:
dataset = load_dataset("csv", data_files={"train": "data/distilbert_train.csv", "test": "data/distilbert_test.csv"}, sep="|")
print(dataset['test'][0])

id2label = {0: "category0", 1: "category1", 2:"category2", 3:"category3", 4:"category4"}
label2id = {"category0": 0, "category1": 1, "category2":2, "category3":3, "category4":4}

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [2]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Map:   0%|          | 0/24997 [00:00<?, ? examples/s]

Map:   0%|          | 0/12005 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.205300,0.307767,0.899375


Epoch,Training Loss,Validation Loss,Accuracy
1,0.205300,0.307767,0.899375
2,0.134400,0.277350,0.917035


TrainOutput(global_step=3126, training_loss=0.2001179532217659, metrics={'train_runtime': 6290.36, 'train_samples_per_second': 7.948, 'train_steps_per_second': 0.497, 'total_flos': 210769494236460.0, 'train_loss': 0.2001179532217659, 'epoch': 2.0})

In [4]:
text = 'what are the best days to run this week?'
# tokenizer = AutoTokenizer.from_pretrained("my_model")
inputs = trainer.tokenizer(text, return_tensors="pt")

# model = AutoModelForSequenceClassification.from_pretrained("my_model")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]